In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import evosax

import json
import math
import drawsvg

In [3]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.opt

import xylo.tuning
import xylo.database

In [4]:
tuning = xylo.tuning.yamaha_YX500R

In [5]:
def speed_of_sound(temp):
  return 331.3 * math.sqrt(1 + temp / 273.15)
  # return 331.3 * math.sqrt((1 + (temp + 273.15)) / 273.15)

speed_of_sound(25)

346.1292013525034

In [6]:
correction = 0.61
radius = 0.018 # m

In [8]:
def freq_of_length(length, temp = 20):
  return speed_of_sound(temp) / (4 * (length + correction * radius))

# f = c / [ 4(L+0.61r)]
# f = c * [1 / 4(L+0.61r)]
# f / [1 / 4(L+0.61r)] = c
# f * 4(L+0.61r) = c
# 4(L+0.61r) = c / f
# L + 0.61r = c / f / 4
# L = (c / f / 4) - 0.61r
def length_of_freq(freq, temp = 20):
  return (speed_of_sound(temp) * (1/freq) * 1/4) - 0.61 * radius


In [9]:
print(freq_of_length(0.1695))
print(freq_of_length(0.300))

475.419191437511
275.9137425900122


In [ ]:
print(length_of_freq(442.23826714, temp = 15))
print(length_of_freq(442.23826714, temp = 20))
print(length_of_freq(442.23826714, temp = 25))

print(length_of_freq(264.33415536, temp = 15))
print(length_of_freq(264.33415536, temp = 20))
print(length_of_freq(264.33415536, temp = 25))


0.18137958633366022
0.18304132751998822
0.1846889566864918
0.31084284582977517
0.31362298425598806
0.3163795128873014
0.28973288017512433


In [25]:
slack = 0.05
tube_lengths = { k: length_of_freq(tuning.note_to_freq(k)) + slack for k in tuning.lengths.keys() }

In [26]:
sum([v for v in tube_lengths.values() if v > 0.10])

4.304979930457928

In [27]:
tube_lengths

{45: 0.283603330175053,
 46: 0.2698759226623779,
 47: 0.25691897533145014,
 48: 0.24468924557501776,
 49: 0.23314591780688237,
 50: 0.22225046724367398,
 51: 0.2119665313319683,
 52: 0.2022597883916446,
 53: 0.19309784307046862,
 54: 0.18445011822761642,
 55: 0.17628775288530885,
 56: 0.16858350590798143,
 57: 0.1613116650875265,
 58: 0.15444796133118893,
 59: 0.14796948766572507,
 60: 0.14185462278750888,
 61: 0.1360829589034412,
 62: 0.130635233621837,
 63: 0.12549326566598415,
 64: 0.12063989419582229,
 65: 0.1160589215352343,
 66: 0.11173505911380821,
 67: 0.10765387644265442,
 68: 0.10380175295399072,
 69: 0.10016583254376325,
 70: 0.09673398066559448,
 71: 0.09349474383286255,
 72: 0.09043731139375444,
 73: 0.08755147945172059,
 74: 0.0848276168109185,
 75: 0.0822566328329921,
 76: 0.07982994709791115,
 77: 0.07753946076761715,
 78: 0.0753775295569041,
 79: 0.07333693822132722,
 80: 0.07141087647699536,
 81: 0.06959291627188163,
 82: 0.06787699033279723,
 83: 0.06625737191643127,

In [28]:
for k, v in tube_lengths.items():
  if v >= 0.08:
    print(f"{k}: {v}m ({tuning.note_to_freq(k)}Hz)")

45: 0.283603330175053m (350.8156324849721Hz)
46: 0.2698759226623779m (371.6762155421418Hz)
47: 0.25691897533145014m (393.77723341802994Hz)
48: 0.24468924557501776m (417.19244620530856Hz)
49: 0.23314591780688237m (442.0Hz)
50: 0.22225046724367398m (468.28268770680853Hz)
51: 0.2119665313319683m (496.1282253527429Hz)
52: 0.2022597883916446m (525.6295448312027Hz)
53: 0.19309784307046862m (556.885104053534Hz)
54: 0.18445011822761642m (589.9992155431552Hz)
55: 0.17628775288530885m (625.0823945689081Hz)
56: 0.16858350590798143m (662.2517279794932Hz)
57: 0.1613116650875265m (701.6312649699441Hz)
58: 0.15444796133118893m (743.3524310842836Hz)
59: 0.14796948766572507m (787.5544668360599Hz)
60: 0.14185462278750888m (834.384892410617Hz)
61: 0.1360829589034412m (884.0Hz)
62: 0.130635233621837m (936.5653754136171Hz)
63: 0.12549326566598415m (992.2564507054858Hz)
64: 0.12063989419582229m (1051.2590896624054Hz)
65: 0.1160589215352343m (1113.770208107068Hz)
66: 0.11173505911380821m (1179.9984310863103H

In [29]:
def mkres(lens, filename, offsetx = 1.3):
  d = drawsvg.Drawing('297mm', '210mm')
  d.append(drawsvg.Rectangle(0, 0, '297mm', '210mm', stroke='green', stroke_width=2, fill='none'))
  d.append(drawsvg.Text(filename, '5mm', '20mm', '8mm'))
  grp = drawsvg.Group()
  y = 10
  for k,v in lens.items():
    mm = v * 1000 + offsetx
    print(k,v)
    grp.append(drawsvg.Text(f'{k}', '2mm', f'{mm-1}mm', f'{y + 3}mm'))
    grp.append(drawsvg.Circle(f'{mm}mm', f'{y + 1}mm', '0.5mm'))
    grp.append(drawsvg.Rectangle(f'{offsetx}mm', f'{y}mm', f'{mm}mm', f'0.5mm', stroke='red', stroke_width=1))
  grp.append(drawsvg.Circle(f'{offsetx}mm', f'{y+1}mm', '1mm'))

  d.append(grp)

  d.save_html(f'data/layout/{filename}.html')
  return d

mkres(tube_lengths, 'resonator-lengths')

45 0.283603330175053
46 0.2698759226623779
47 0.25691897533145014
48 0.24468924557501776
49 0.23314591780688237
50 0.22225046724367398
51 0.2119665313319683
52 0.2022597883916446
53 0.19309784307046862
54 0.18445011822761642
55 0.17628775288530885
56 0.16858350590798143
57 0.1613116650875265
58 0.15444796133118893
59 0.14796948766572507
60 0.14185462278750888
61 0.1360829589034412
62 0.130635233621837
63 0.12549326566598415
64 0.12063989419582229
65 0.1160589215352343
66 0.11173505911380821
67 0.10765387644265442
68 0.10380175295399072
69 0.10016583254376325
70 0.09673398066559448
71 0.09349474383286255
72 0.09043731139375444
73 0.08755147945172059
74 0.0848276168109185
75 0.0822566328329921
76 0.07982994709791115
77 0.07753946076761715
78 0.0753775295569041
79 0.07333693822132722
80 0.07141087647699536
81 0.06959291627188163
82 0.06787699033279723
83 0.06625737191643127
84 0.06472865569687723
85 0.0632857397258603
86 0.061923808405459246
87 0.06063831641649605
88 0.059424973548955574
